# Checking lambda operator explicitly

In [1]:
# import external modules
import networkx as nx
import numpy as np
import scipy
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt

# import custom modules for linking number identification
import intertwined.sampling as itws
import intertwined.edgePriority as itwe
import kirchhoff.circuit_dual as kcd
import intertwined.cycleToolsLinking as ctl
# import local module to create entangled toy models
import tangledGenerators as tg
import layoutPlots

In [2]:
def shrinkSystem(D, factor):
    
    gs = []
    for d in D.layer:
        G = nx.Graph(d.G)
        
        for n in G.nodes():
            p = G.nodes[n]['pos']
            G.nodes[n]['pos'] = np.array(p)*factor
        
        gs.append(G)
        
    DN = kcd.initialize_dual_circuit_from_networkx(*gs, [])
    
    return DN

def check_numeric_invariance(iteration, dual_circuit):
    
    graph_sets = [k.G for k in dual_circuit.layer]
    
    mat_sets = []
    for i in range(iteration):
        
        P, lk_mat = itwe.getEdgeLinkageOperator(graph_sets)


#         cyc_nx_base=[itws.calc_cycle_basis(G) for G in graph_sets]
#         T = ctl.linkedCycles_fastExtraTools()
#         bool_res,numeric_res = T.calc_linkage_cycleSets_nxGraph(*cyc_nx_base)
#         lk_mat = itws.extract_linkage_matrix(numeric_res, [len(cb) for cb in cyc_nx_base])
#         graph_matrices = itws.get_basis_matrices(graph_sets,cyc_nx_base)
#         cyc_mat_dir = [cm for cm in graph_matrices[-2]]
#         cyc_mat_inv = [scipy.linalg.pinv(cm) for cm in cyc_mat_dir]
#         P = np.dot(np.dot(cyc_mat_inv[0].T,lk_mat),cyc_mat_inv[1])
        
        
        mat_sets.append(P)
        
    return mat_sets

def diffMatrixSets(mat_sets):
    
    matSum = mat_sets[0]
    for ms in mat_sets[1:]:
        matSum = np.add(matSum,ms)    
    matSum = matSum/len(mat_sets)     
    
    matDiffSets = [np.subtract(ms,matSum) for ms in mat_sets]
    
    matDiff=[ np.linalg.norm(mds) for mds in matDiffSets]
    diffMean = np.mean(matDiff)
    diffStd = np.std(matDiff)
        
    return diffMean, diffStd  

In [3]:
import os
path_in = '/Users/kramer/Documents/phd/projects/insilico/output/cluster_linkage_project/eval/regular_lattices/43616819/data'
files = [f for f in os.listdir(path_in) if '.npy' in f]
P = [np.load(path_in+'/'+f) for f in files]
diffMatrixSets(P)

(1.7573994172698832e-15, 9.399375638892348e-16)

In [4]:
path_in = '/Users/kramer/Documents/phd/projects/insilico/output/cluster_linkage_project/eval/regular_lattices/43616870/data'
files = [f for f in os.listdir(path_in) if '.npy' in f]
P = [np.load(path_in+'/'+f) for f in files]
diffMatrixSets(P)

(1.1989197536832115e-15, 8.920195325386129e-16)

In [5]:
path_in = '/Users/kramer/Documents/phd/projects/insilico/output/cluster_linkage_project/eval/regular_lattices/43616921/data'
files = [f for f in os.listdir(path_in) if '.npy' in f]
P = [np.load(path_in+'/'+f) for f in files]
diffMatrixSets(P)

(0.839432271823091, 0.5427068007905909)

# test non-least square solution for different cycle bases,
# by reevaluating lambda repeatedly and measuring the mean difference in all elements

In [6]:
# # test dual ladder
# D = tg.createLabelCatenation(5)
# # tg.shiftLadder(D, [0.495, 0., 0.])
# # D = shrinkSystem(D, 1.)
# mat_sets = check_numeric_invariance(20, D)
# diffMatrixSets(mat_sets)

In [7]:
# for i, p in enumerate(mat_sets):
#     if i ==0 :
#         layoutPlots.update_priorityPlot(D, p)

In [8]:
# graph_sets = [k.G for k in D.layer]
# cyc_nx_base=[itws.calc_cycle_basis(G) for G in graph_sets]

# # # compute linkage of basis vectors
# numeric_res= itws.calc_basis_linkage(*cyc_nx_base)
# data = [np.round(v,2) for k,v in numeric_res.items()]

# plt.hist(data, bins=50)
# plt.yscale('log')
# plt.show()

In [9]:
# # test dual laves
# D = tg.createLabelLaves(2)
# mat_sets = check_numeric_invariance(10, D)
# diffMatrixSets(mat_sets)

In [10]:
# for i, p in enumerate(mat_sets):
#     if i ==0 :
#         layoutPlots.update_priorityPlot(D, p)

In [11]:
# graph_sets = [k.G for k in D.layer]
# cyc_nx_base=[itws.calc_cycle_basis(G) for G in graph_sets]

# # # compute linkage of basis vectors
# numeric_res= itws.calc_basis_linkage(*cyc_nx_base)
# data = [np.round(v,2) for k,v in numeric_res.items()]
# plt.hist(data, bins=50)
# plt.yscale('log')

In [12]:
# # check lambda operator explicitly for linkage number calculation of closed curves
# import intertwined.edge_algebra as ea

# D = tg.createLabelCatenation(2)
# graph_sets = [k.G for k in D.layer]
# cyc_nx_base,lk_mat = itws.sample_intertwinedness_basisBonanza_beta(graph_sets)
# graph_matrices=itws.get_basis_matrices(graph_sets,cyc_nx_base)
# cyc_mat_dir = [cm for cm in graph_matrices[-2]]
# cyc_mat_inv = [scipy.linalg.pinv(cm) for cm in cyc_mat_dir]
# P=np.dot(np.dot(cyc_mat_inv[0].T,lk_mat),cyc_mat_inv[1])

# c1 = cyc_nx_base[0][0]
# c2 = cyc_nx_base[1][0]
# ev1 = ea.edge_column_rand(c1,graph_matrices[0][0], graph_matrices[1][0])
# ev2 = ea.edge_column_rand(c2,graph_matrices[0][1], graph_matrices[1][1])
# print(ev1)
# print(ev2)

# lk=np.dot(np.dot(ev1.T, P), ev2)
# print(lk)

In [13]:
%%timeit -r 1
# generate basis vectors and corresponding graph matrices
D = tg.createLabelLaves(3)
graph_sets = [k.G for k in D.layer]
print('calc basis')
cyc_nx_bases=[itws.calc_cycle_minimum_basis(G) for G in graph_sets]
# cyc_nx_bases=[itws.calc_cycle_basis(G) for G in graph_sets]
# for l in cyc_nx_bases[0]:
#     print(len(l))
# compute linkage of basis vectors
print('calc linking')
numeric_res = itws.calc_basis_linkage(*cyc_nx_bases)

circuit(): initialized and ready for (some) action :)
circuit(): initialized and ready for (some) action :)
calc basis
calc linking
circuit(): initialized and ready for (some) action :)
circuit(): initialized and ready for (some) action :)
calc basis
calc linking
5min 17s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
